# Approach and Refferences

> efficientnetb3a and efficientnetbv2m with aux loss for study + yolov5m and yolov5x for image 

## Refferences

1. henhttps://www.kaggle.com/c/siim-covid19-detection/discussion/246586gk's aux loss https://www.kaggle.com/c/siim-covid19-detection/discussion/240233
2. alien's 2 class tricks https://www.kaggle.com/c/siim-covid19-detection/discussion/246586
3. darian's duplicate analysis https://www.kaggle.com/c/siim-covid19-detection/discussion/240878

## Training notebook

1. drop duplicate and create mask https://www.kaggle.com/drzhuzhe/siiim-covid-stratified-k-fold-and-create-mask

2. training study level https://www.kaggle.com/drzhuzhe/covid19-classify 

    efficientnetb3a 1e-3 10 ep + 1e-4 5ep CV map*0.66 score (3.76 + 3.92 + 3.85 + 3.7 + 3.6)/5 avg 3.766
    
    efficientnetv2m ReduceLROnPlateau CV map*0.66 score (3.92 + 3.92 + 3.81 + 3.85 + 3.74)

3. training yolov5m https://www.kaggle.com/drzhuzhe/covid19-det?scriptVersionId=67605495 
    
    yolom only 10 epoch  CV map score (0.4947 + 0.5103 + 0.4848 +0.4692 +0.5198)/5 avg 0.49575 
    
    yolox 30 epoch CV map score (0.525 + 0.525 + 0.488 + 0.488+ 0.481)
    

4. ~2 class https://www.kaggle.com/drzhuzhe/siim-covid19-efnb7-train-fold0-5-2class  yolov5m 15 epoch (0.869 + 0.860 + 0.882 + 0.878 + 0.876)/5 avg 0.872~ Only use 'negative' probility as 'none'


In [ ]:
with_ensemble = True
with_study = True
with_image = True
with_2class = True

In [ ]:
!conda install '../input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -y --offline
!conda install '../input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -y --offline

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

import cv2
import sys
import math

from timeit import default_timer as timer
from datetime import datetime
from numba import cuda

# read dicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image
import csv

# Preporcess

In [ ]:
_SIZE_ = 512 
_Type_ = "png"

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data


def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
study = []

    
#for split in ['test', 'train']:
for split in ['test']:
    save_dir = f'/kaggle/tmp/{split}/'
    
    os.makedirs(save_dir, exist_ok=True)
    
    for study_id in tqdm(os.listdir(f'../input/siim-covid19-detection/{split}')):
        for dirname, _, filenames in os.walk(f'../input/siim-covid19-detection/{split}/{study_id}'):
            for file in filenames:
                # set keep_ratio=True to have original aspect ratio
                xray = read_xray(os.path.join(dirname, file))

                im = resize(xray, size=_SIZE_)
                _name = file.replace('dcm', _Type_)
                im.save(os.path.join(save_dir, _name))

                _img_id = file.replace('.dcm', '')
                image_id.append(_img_id)

                _w = xray.shape[0]
                _h = xray.shape[1]
                dim0.append(_w)
                dim1.append(_h)
                splits.append(split)
                study.append(study_id)

In [ ]:
meta_df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits, 'study': study})
meta_df.to_csv('/kaggle/tmp/meta.csv', index=False)

In [ ]:
meta_df

# Config

In [ ]:
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
_det_model_path = "/kaggle/input/collect-submit-model/det/"


_classify_model1_path = "/kaggle/input/covidmodels/Archive/classify-ep12/"

_classify_model2_path = "/kaggle/input/covidmodels/"

#_test_files_path = "/kaggle/input/covid19512/test/"
#_data_dir = "/kaggle/input/covid19512/"

_test_files_path = "/kaggle/tmp/test/"
_data_dir = "/kaggle/tmp/"

meta_df = pd.read_csv(_data_dir + "meta.csv")

In [ ]:
IMG_SIZE = 512

# Clssification

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn.parallel.data_parallel import data_parallel

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import collections
from collections import defaultdict

import timm
from timm.models.efficientnet import *

import torch.cuda.amp as amp

In [ ]:
data_dir = _data_dir
image_size = IMG_SIZE

study_name_to_predict_string = {
    'Negative for Pneumonia'  :'negative',
    'Typical Appearance'      :'typical',
    'Indeterminate Appearance':'indeterminate',
    'Atypical Appearance'     :'atypical',
}

study_name_to_label = {
    'Negative for Pneumonia'  :0,
    'Typical Appearance'      :1,
    'Indeterminate Appearance':2,
    'Atypical Appearance'     :3,
}
study_label_to_name = { v:k for k,v in study_name_to_label.items()}
num_study_label = len(study_name_to_label)

In [ ]:
def make_fold(mode='train-0'):
    if 'test' in mode:
        df_meta  = pd.read_csv(data_dir+'meta.csv')
        df_valid = df_meta[df_meta['split']=='test'].copy()

        for l in study_name_to_label.keys():
            df_valid.loc[:,l]=0
        df_valid = df_valid.reset_index(drop=True)
        return df_valid

In [ ]:
class SiimDataset(Dataset):
    def __init__(self, df, augment=None):
        super().__init__()
        self.df = df
        self.augment = augment
        self.length = len(df)

    def __str__(self):
        string  = ''
        string += '\tlen = %d\n'%len(self)
        string += '\tdf  = %s\n'%str(self.df.shape)

        string += '\tlabel distribution\n'
        for i in range(num_study_label):
            n = self.df[study_label_to_name[i]].sum()
            string += '\t\t %d %26s: %5d (%0.4f)\n'%(i, study_label_to_name[i], n, n/len(self.df) )
        return string


    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]

        image_file = data_dir + '/test/%s.png' % (d.image_id)
        image = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
        onehot = d[study_name_to_label.keys()].values

        mask = np.zeros_like(image)


        r = {
            'index' : index,
            'd' : d,
            'image' : image,
            'mask' : mask,
            'onehot' : onehot,
        }
        if self.augment is not None: r = self.augment(r)
        return r

def null_collate(batch):
    collate = defaultdict(list)

    for r in batch:
        for k, v in r.items():
            collate[k].append(v)

    # ---
    batch_size = len(batch)
    onehot = np.ascontiguousarray(np.stack(collate['onehot'])).astype(np.float32)
    collate['onehot'] = torch.from_numpy(onehot)

    image = np.stack(collate['image'])
    image = image.reshape(batch_size, 1, image_size,image_size).repeat(3,1)
    image = np.ascontiguousarray(image)
    image = image.astype(np.float32) / 255
    collate['image'] = torch.from_numpy(image)


    mask = np.stack(collate['mask'])
    mask = mask.reshape(batch_size, 1, image_size,image_size)
    mask = np.ascontiguousarray(mask)
    mask = mask.astype(np.float32) / 255
    collate['mask'] = torch.from_numpy(mask)

    return collate

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        e = efficientnet_b3a(pretrained=False, drop_rate=0.3, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1536,num_study_label)
        self.mask = nn.Sequential(
            nn.Conv2d(136, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1     # ; print('input ',   x.shape)

        x = self.b0(x) #; print (x.shape)  # torch.Size([2, 40, 256, 256])
        x = self.b1(x) #; print (x.shape)  # torch.Size([2, 24, 256, 256])
        x = self.b2(x) #; print (x.shape)  # torch.Size([2, 32, 128, 128])
        x = self.b3(x) #; print (x.shape)  # torch.Size([2, 48, 64, 64])
        x = self.b4(x) #; print (x.shape)  # torch.Size([2, 96, 32, 32])
        x = self.b5(x) #; print (x.shape)  # torch.Size([2, 136, 32, 32])
        #------------
        mask = self.mask(x)
        #-------------
        x = self.b6(x) #; print (x.shape)  # torch.Size([2, 232, 16, 16])
        x = self.b7(x) #; print (x.shape)  # torch.Size([2, 384, 16, 16])
        x = self.b8(x) #; print (x.shape)  # torch.Size([2, 1536, 16, 16])
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.5, training=self.training)
        logit = self.logit(x)
        return logit, mask

In [ ]:
class NetV2(nn.Module):
    def __init__(self):
        super(NetV2, self).__init__()

        #e = efficientnet_b3a(pretrained=True, drop_rate=0.3, drop_path_rate=0.2)
        #e = efficientnetv2_rw_m(pretrained=True, drop_rate=0.5, drop_path_rate=0.2)
        #e = tf_efficientnetv2_m_in21ft1k(pretrained=True, drop_rate=0.3, drop_path_rate=0.2)
        e = tf_efficientnetv2_m(pretrained=False, drop_path_rate=0.4)
        
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1280,num_study_label)
        self.mask = nn.Sequential(
            nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            #nn.Conv2d(128, 128, kernel_size=3, padding=1),
            #nn.BatchNorm2d(128),
            #nn.ReLU(inplace=True),

            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1     # ; print('input ',   x.shape)
        """
        rw_m
        torch.Size([2, 32, 256, 256])
        torch.Size([2, 32, 256, 256])
        torch.Size([2, 56, 128, 128])
        torch.Size([2, 80, 64, 64])
        torch.Size([2, 152, 32, 32])
        torch.Size([2, 192, 32, 32])
        torch.Size([2, 328, 16, 16])
        torch.Size([2, 2152, 16, 16])
        """
        x = self.b0(x) ; #print (x.shape)  # torch.Size([2, 32, 256, 256])
        x = self.b1(x) ; #print (x.shape)  # torch.Size([2, 32, 256, 256])
        x = self.b2(x) ; #print (x.shape)  # torch.Size([2, 56, 128, 128])
        x = self.b3(x) ; #print (x.shape)  # torch.Size([2, 80, 64, 64])
        x = self.b4(x) ; #print (x.shape)  # torch.Size([2, 152, 32, 32])
        x = self.b5(x) ; #print (x.shape)  # torch.Size([2, 192, 32, 32])

        #------------
        mask = self.mask(x)
        #-------------

        x = self.b6(x) ; #print (x.shape)  # torch.Size([2, 328, 16, 16])
        
        x = self.b7(x) ; #print (x.shape)  # torch.Size([2, 512, 16, 16])
        
        x = self.b8(x) ; #print (x.shape)  # torch.Size([2, 2152, 16, 16])
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        x = F.dropout(x, 0.5, training=self.training)

        logit = self.logit(x)
        return logit, mask

In [ ]:
def probability_to_df_study(df_valid, probability):
    df_study = pd.DataFrame()
    df_image = df_valid.copy()
    df_study.loc[:,'id'] = df_valid.study + '_study'
    for i in range(num_study_label):
        df_study.loc[:,study_name_to_predict_string[study_label_to_name[i]]]=probability[:,i]
        df_image.loc[:,study_name_to_predict_string[study_label_to_name[i]]]=probability[:,i]
    
    
    df_study = df_study.groupby('id', as_index=False).mean()
    df_study.loc[:, 'PredictionString'] = \
           'negative '      + df_study.negative.apply(lambda x: '%0.6f'%x)      + ' 0 0 1 1' \
        + ' typical '       + df_study.typical.apply(lambda x: '%0.6f'%x)       + ' 0 0 1 1' \
        + ' indeterminate ' + df_study.indeterminate.apply(lambda x: '%0.6f'%x) + ' 0 0 1 1' \
        + ' atypical '      + df_study.atypical.apply(lambda x: '%0.6f'%x)      + ' 0 0 1 1'

    df_study = df_study[['id','PredictionString']]
    return df_study, df_image

In [ ]:
def do_predict(net, valid_loader, tta=['flip','scale']): #flip

    valid_probability = []
    valid_num = 0

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image  = batch['image'].cuda()
        onehot = batch['onehot']
        label  = onehot.argmax(-1)

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(F.softmax(logit,-1))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(F.softmax(logit,-1))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                logit, mask = net(F.interpolate(image, scale_factor=1.33, mode='bilinear', align_corners=False))
                probability.append(F.softmax(logit,-1))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        valid_num += batch_size
        valid_probability.append(probability.data.cpu().numpy())
        print('\r %8d / %d  %s' % (valid_num, len(valid_loader.dataset), time_to_str(timer() - start_timer, 'sec')),
              end='', flush=True)

    assert(valid_num == len(valid_loader.dataset))
    print('')

    probability = np.concatenate(valid_probability)
    return probability

In [ ]:
class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout  #stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1 ):
        if '\r' in message: is_file=0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()
            #time.sleep(1)

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass

In [ ]:
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError

In [ ]:
netList = [Net, NetV2]

if with_ensemble:
    VersionList = [0 , 1]
else:
    VersionList = [0]

def run_submit():
    for fold in [0 ,1 ,2, 3, 4]:
        out_dir = './study_predict/'
        
        initial_checkpoint_v1 = \
                _classify_model1_path + 'f' + str(fold) + '.pth'

        # v2
        initial_checkpoint_v2 = _classify_model2_path + 'effv2-m-f' + str(fold) + '.pth'
           
        initial_checkpoint = [initial_checkpoint_v1, initial_checkpoint_v2]  
        
        
        for ver in VersionList:
            

            ## setup  ----------------------------------------
            #mode = 'local'
            mode = 'remote'

            submit_dir = out_dir + 'v' + str(ver)+ '/%s-fold%d'%(mode, fold)
            os.makedirs(submit_dir, exist_ok=True)
        
        

            log = Logger()
            log.open(out_dir + 'log.submit.txt', mode='a')
            log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
            #log.write('\t%s\n' % COMMON_STRING)
            log.write('\n')

            #
            ## dataset ------------------------------------

            df_valid = make_fold('test')

            valid_dataset = SiimDataset(df_valid)
            valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 32,#128, #
                drop_last   = False,
                num_workers = 8,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
            log.write('mode : %s\n'%(mode))
            log.write('valid_dataset : \n%s\n'%(valid_dataset))

            ## net -------------------------------------       
            if 1:
                #V2
                net = netList[ver]().cuda()
                net.load_state_dict(torch.load(initial_checkpoint[ver])['state_dict'], strict=True)

                #---
                start_timer = timer()
                probability = do_predict(net, valid_loader)
                log.write('time %s \n' % time_to_str(timer() - start_timer, 'min'))
                log.write('probability %s \n' % str(probability.shape))

                np.save(submit_dir+ '/probability.npy',probability)
                df_valid.to_csv(submit_dir + '/df_valid.csv', index=False)

            else:
                probability = np.load(submit_dir + '/probability.npy')

            #----
            df_study, df_image = probability_to_df_study(df_valid, probability)

            #df_image = probability_to_df_image(df_valid, None, None)
            #df_submit = pd.concat([df_study,df_image])
            df_submit = pd.concat([df_study])
            df_submit.to_csv(submit_dir + '/submit.csv', index=False)

            log.write('submit_dir : %s\n' % (submit_dir))
            log.write('initial_checkpoint : %s\n' % (initial_checkpoint))
            log.write('df_submit : %s\n' % str(df_submit.shape))
            log.write('%s\n' % str(df_submit))
            log.write('\n')
        

In [ ]:
IDENTIFIER   = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
if with_study:
    run_submit()

In [ ]:
#df_valid = make_fold('test')
#df_valid

In [ ]:
def run_remote_ensemble():
    out_dir = './study_predict/'
    log = Logger()
    log.open(out_dir + 'log.submit.txt', mode='a')
    log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
    #log.write('\t%s\n' % COMMON_STRING)
    log.write('\n')


    submit_dir=[
        out_dir+'v0/remote-fold0',
        out_dir+'v0/remote-fold1',
        out_dir+'v0/remote-fold2',
        out_dir+'v0/remote-fold3',
        out_dir+'v0/remote-fold4'    
    ]
    
    if with_ensemble:
         submit_dir += [
            out_dir+'v1/remote-fold0',
            out_dir+'v1/remote-fold1',
            out_dir+'v1/remote-fold2',
            out_dir+'v1/remote-fold3',
            out_dir+'v1/remote-fold4'
         ] 

    probability=0
    for d in submit_dir:
        p = np.load(d + '/probability.npy')
        probability += p**0.5
    probability = probability/len(submit_dir)


    #----
    df_valid = pd.read_csv(submit_dir[0] + '/df_valid.csv')

    df_study, df_image  = probability_to_df_study(df_valid, probability)
    #df_image  = probability_to_df_image(df_valid, None, None)
    #df_submit = pd.concat([df_study, df_image])
    df_submit = pd.concat([df_study])
    
    #df_submit.to_csv(out_dir + '/effb3-full-512-mask-submit-ensemble1.csv', index=False)

    log.write('submit_dir : %s\n' % (submit_dir))
    log.write('df_submit : %s\n' % str(df_submit.shape))
    log.write('%s\n' % str(df_submit))
    log.write('\n')
    
    return df_submit, df_image

In [ ]:
IDENTIFIER   = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
if with_study:
    predict_study, df_image = run_remote_ensemble()

In [ ]:
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

In [ ]:
submit_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

In [ ]:
df_2class = meta_df.loc[meta_df.split == "test"].copy()

In [ ]:
#test_paths = f'/kaggle/input/covid19512/test/' + df_2class['image_id'] +'.png'

In [ ]:
df_2class['none'] = 0

In [ ]:
#label_cols = df_2class.columns[5]
#label_cols

In [ ]:
if with_2class:
    df_2class['none'] = df_image['negative'].values
    df_2class.head()

# Detect

In [ ]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

In [ ]:
MODEL_PATH = _det_model_path + "yolom-f0.pt" + " " + _det_model_path + "yolom-f1.pt" + " " + _det_model_path + "yolom-f2.pt" + " " + _det_model_path + "yolom-f3.pt" + " " + _det_model_path + "yolom-f4.pt"
_yolox_path = "/kaggle/input/covidmodels/yolox-v0/"

#MODEL_PATH = _yolox_path + "f0.pt" + " " + _yolox_path + "f1.pt" + " " + _yolox_path + "f2.pt" + " " + _yolox_path + "f3.pt" + " " + _yolox_path + "f4.pt"

if with_ensemble:
    MODEL_PATH += " " + _yolox_path + "f0.pt" + " " + _yolox_path + "f1.pt" + " " + _yolox_path + "f2.pt" + " " + _yolox_path + "f3.pt" + " " + _yolox_path + "f4.pt"

In [ ]:
if with_image:
    if with_ensemble:
        !python detect.py --weights {MODEL_PATH} \
                          --source {_test_files_path} \
                          --img {IMG_SIZE} \
                          --conf 0.0001 \
                          --iou-thres 0.5 \
                          --save-txt \
                          --save-conf \
                          --augment
    else:
        !python detect.py --weights {MODEL_PATH} \
                          --source {_test_files_path} \
                          --img {IMG_SIZE} \
                          --conf 0.001 \
                          --iou-thres 0.5 \
                          --save-txt \
                          --save-conf 

In [ ]:
if with_image:
    def yolo2voc(image_height, image_width, bboxes):

        bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

        bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
        bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

        bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
        bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

        return bboxes


    image_ids = []
    PredictionStrings = []

    #for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
    for dir_path, _, filenames in os.walk(_test_files_path):
            print(len(filenames))
    for file in filenames:
        file_path = 'runs/detect/exp/labels/' + file.replace(".png", '.txt')
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = meta_df.loc[meta_df.image_id == image_id,['dim1', 'dim0']].values[0]
        if not os.path.exists(file_path):
            bboxes = "none 1 0 0 1 1"
        else:
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
            for idx in range(len(bboxes)):
                bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
            bboxes = ' '.join(bboxes)
        image_id += "_image"
        image_ids.append(image_id)
        PredictionStrings.append(bboxes)

    predict_image = pd.DataFrame({'id':image_ids,
                            'PredictionString':PredictionStrings})

# Submit

In [ ]:
if with_image:
    for i in range(predict_image.shape[0]):
        if predict_image.loc[i,'PredictionString'] == "none 1 0 0 1 1":
            continue
        sub_df_split = predict_image.loc[i,'PredictionString'].split()
        sub_df_list = []
        for j in range(int(len(sub_df_split) / 6)):
            sub_df_list.append('opacity')
            sub_df_list.append(sub_df_split[6 * j + 1])
            sub_df_list.append(sub_df_split[6 * j + 2])
            sub_df_list.append(sub_df_split[6 * j + 3])
            sub_df_list.append(sub_df_split[6 * j + 4])
            sub_df_list.append(sub_df_split[6 * j + 5])
        predict_image.loc[i,'PredictionString'] = ' '.join(sub_df_list)

In [ ]:
if with_image and with_2class:
    for i in range(predict_image.shape[0]):
        if predict_image.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
            _none = str(df_2class.loc[df_2class.image_id + "_image" == predict_image.iloc[i].id]['none'].item())
            predict_image.loc[i,'PredictionString'] = predict_image.loc[i,'PredictionString'] + ' none ' + _none + ' 0 0 1 1'

In [ ]:
if with_study:
    for index, row in predict_study.iterrows():
        submit_df.loc[submit_df.id == row.id, "PredictionString"] = row.PredictionString

if with_image:
    for index, row in predict_image.iterrows():
        submit_df.loc[submit_df.id == row.id, "PredictionString"] = row.PredictionString

In [ ]:
submit_df.to_csv('/kaggle/working/submission.csv',index = False)  

In [ ]:
"""
sample = submit_df.iloc[-1]
print(sample.id, sample.PredictionString)

_study = meta_df.loc[meta_df.image_id + "_image" == sample.id].study.item()
print(_study)
_study_item = submit_df.loc[submit_df.id == _study + "_study"]
_study_item
"""

In [ ]:
if with_image:
    shutil.rmtree('/kaggle/working/yolov5')
if with_study:
    shutil.rmtree('/kaggle/working/study_predict')